In [ ]:
# draw function on 2d elements

import webgpu.jupyter as wj
from webgpu.colormap import Colorbar
from ngsolve import *
from ngsolve_webgpu.mesh import MeshData, PointNumbers
from ngsolve_webgpu.cf import FunctionData, CFRenderer

def draw_func_with_vert_numbers(mesh, cf, order=2):
    mesh_data = MeshData(mesh.ngmesh)
    function_data = FunctionData(mesh_data, cf, order)
    cfrenderer = CFRenderer(function_data)
    point_numbers = PointNumbers(mesh_data, 13)
    cfrenderer.colormap.set_min_max(-1,1)
    wj.Draw([ cfrenderer, Colorbar(cfrenderer.colormap), point_numbers])


from netgen.occ import *

box = Box(Pnt(0,0,0), Pnt(1,1,1))
cyl = Cylinder(Pnt(0,0.0,0.0), X, r=0.3, h=0.5)
geo = OCCGeometry(cyl)
m = Mesh(geo.GenerateMesh(maxh=6))
m.Curve(5)

cf=sin(20*x)

draw_func_with_vert_numbers(m, cf, order=5)
ngm = m.ngmesh


In [ ]:
# draw geometry

import webgpu.jupyter as wj
from netgen.occ import *
from ngsolve import *
import webgpu.jupyter

box = Box((-1,-1,-1), (1,1,1)) + Sphere((1,1,1), 0.4)

def draw_geo(geo):
    import ngsolve_webgpu.geometry
    if isinstance(geo, TopoDS_Shape):
        geo = OCCGeometry(geo)
    obj = ngsolve_webgpu.geometry.GeometryRenderer(geo)
    wj.Draw([obj])

draw_geo(box)

In [ ]:
# draw isosurface
import webgpu.jupyter
from netgen.occ import *
from ngsolve import *
import ngsolve_webgpu.cf
import numpy as np

box = Box((-1,-1,-1), (1,1,1))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=.2))
gf = GridFunction(H1(mesh, order=2))
cf = 1.2**2 - (x**2 + y**2 + z**2)
gf.Set(cf)

def draw_iso(levelset, func, mesh, order=2):
    import ngsolve_webgpu.isosurface
    import ngsolve_webgpu.mesh
    import ngsolve_webgpu.cf
    meshdata = ngsolve_webgpu.mesh.MeshData(mesh.ngmesh)
    data = ngsolve_webgpu.cf.FunctionData(meshdata, func, order=order)   
    levelset_data = ngsolve_webgpu.cf.FunctionData(meshdata, -levelset, order=order)
    clipping = ngsolve_webgpu.cf.Clipping()
    colormap = ngsolve_webgpu.cf.Colormap()
    clipsol = ngsolve_webgpu.isosurface.NegativeClippingRenderer(data, levelset_data, clipping, colormap)
    neg_outer = ngsolve_webgpu.isosurface.NegativeSurfaceRenderer(data, levelset_data, clipping=clipping, colormap=colormap)
    iso = ngsolve_webgpu.isosurface.IsoSurfaceRenderer(data, levelset_data, clipping, colormap)
    clipping.mode = clipping.Mode.PLANE
    iso.colormap = neg_outer.colormap
    clipsol.colormap = neg_outer.colormap
    iso.clipping = clipping
    scene = webgpu.jupyter.Draw([iso, clipsol, neg_outer, Colorbar(neg_outer.colormap)])
    clipping.add_options_to_gui(scene.gui)
    return scene

scene = draw_iso(gf, x, mesh)

In [ ]:
gf.Set(0.7**2 - (x**2 + y**2 + z**2))
scene.redraw()

Animate time dependent data (or eigenvectors,...)

In [ ]:
import webgpu.jupyter as wj
from ngsolve import *
from ngsolve_webgpu.mesh import MeshData, PointNumbers
from ngsolve_webgpu.cf import FunctionData, CFRenderer
from ngsolve_webgpu.animate import Animation

def draw(cf, mesh):
    md = MeshData(mesh)
    fd = FunctionData(md, cf, order=3)
    cfr = CFRenderer(fd)
    ani = Animation(cfr)
    colormap = cfr.colormap
    colormap.set_min_max(-1,1)
    scene = wj.Draw([ani, Colorbar(colormap)])
    ani.add_options_to_gui(scene.gui)
    ani.add_time()
    return scene, ani

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
gf = GridFunction(H1(mesh, order=3))
t = Parameter(0)
f = sin(10*(x-0.1 * t))
gf.Interpolate(f)
scene, animation = draw(cos(10*(y-0.05*t))*gf*gf, mesh)

In [ ]:
for tval in range(1, 101):
    t.Set(tval)
    gf.Interpolate(f)
    store = tval % 10 == 0
    if store:
        animation.add_time()
    scene.redraw(store)
    

3d volume elements (only tets for now)

In [ ]:
import webgpu.jupyter as wj
from ngsolve import *

def draw(mesh):
    from ngsolve_webgpu.mesh import MeshData, MeshElements3d
    data = MeshData(mesh)
    render = MeshElements3d(data)
    scene = wj.Draw(render)
    render.add_options_to_gui(scene.gui)
    render.clipping.add_options_to_gui(scene.gui)
    return scene

mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))
draw(mesh)

In [ ]:
import webgpu.jupyter as wj
from ngsolve import *


def draw(mesh, cf, order=5):
    from ngsolve_webgpu.mesh import MeshData
    from ngsolve_webgpu.cf import FunctionData, CFRenderer
    from ngsolve_webgpu.clipping import VolumeCF, ClippingCF
    from webgpu.colormap import Colorbar
    mesh_data = MeshData(mesh)
    function_data = FunctionData(mesh_data, cf, order)
    clip = ClippingCF(function_data)
    colormap, clipping = clip.colormap, clip.clipping
    cfr = CFRenderer(function_data)
    clipping.mode = clipping.Mode.PLANE
    cfr.clipping = clipping
    cfr.colormap = colormap
    colormap.set_min_max(0,1)
    clipping.center = [0.7,0.5,0.5]
    clipping.normal = [1,-1,1]
    scene = wj.Draw([cfr, clip, Colorbar(colormap)])
    clipping.add_options_to_gui(scene.gui)


m = Mesh(unit_cube.GenerateMesh(maxh=0.2))
draw(m, sin(10*z) * cos(15*x))
    